In [1]:
# imports
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import warnings
import statistics as st             # for statistical analysis
warnings.filterwarnings("ignore")

In [2]:
# setting up several paths 
sys.path.append(".")
sys.path.append("..")
sys.path.insert(1, '../scripts')
from dataCleaner import *
from dataVisualizer import *

In [3]:
# read data
df = pd.read_csv('../data/Week1_challenge_data_source_filled_2.csv.bz2')
df

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,total_gaming,total_youtube,total_netflix,total_social,fixed_dur,fixed_activity_dur_dl,fixed_activity_dur_ul,fixed_total_ul,total_data,decile
0,1.311450e+19,2019-04-04 12:01:00,770.0,2019-04-25 14:35:00,662.0,86399.0,2.082010e+14,3.366496e+10,3.552120e+13,9.16457E+15,...,2905912.0,2905912.0,2905912.0,2905912.0,86399.0,37624.0,38787.0,36749741.0,345629377.0,"(72290.0, 86399.0]"
1,1.311450e+19,2019-04-09 13:04:00,235.0,2019-04-25 08:15:00,606.0,86399.0,2.082020e+14,3.368185e+10,3.579400e+13,L77566A,...,4414096.0,4414096.0,4414096.0,4414096.0,86399.0,168.0,3560.0,53800391.0,707185356.0,"(72290.0, 86399.0]"
2,1.311450e+19,2019-04-09 17:42:00,1.0,2019-04-25 11:58:00,652.0,86399.0,2.082000e+14,3.376063e+10,3.528150e+13,D42335A,...,10229119.0,10229119.0,10229119.0,10229119.0,86399.0,0.0,0.0,27883638.0,307690973.0,"(72290.0, 86399.0]"
3,1.311450e+19,2019-04-10 00:31:00,486.0,2019-04-25 07:36:00,171.0,86399.0,2.082010e+14,3.375034e+10,3.535660e+13,T21824A,...,11811761.0,11811761.0,11811761.0,11811761.0,86399.0,3330.0,37882.0,43324218.0,889352748.0,"(72290.0, 86399.0]"
4,1.311450e+19,2019-04-12 20:10:00,565.0,2019-04-25 10:40:00,954.0,86399.0,2.082010e+14,3.369980e+10,3.540700e+13,D88865A,...,7748843.0,7748843.0,7748843.0,7748843.0,86399.0,0.0,0.0,38542814.0,607681403.0,"(72290.0, 86399.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1.304240e+19,2019-04-29 07:28:00,615.0,2019-04-30 00:01:00,407.0,59587.0,2.082010e+14,3.366865e+10,3.533250e+13,T85721A,...,1265634.0,1265634.0,1265634.0,1265634.0,59587.0,3380381.0,2355730.0,53745392.0,872761860.0,"(45959.8, 72290.0]"
149996,7.277830e+18,2019-04-29 07:28:00,451.0,2019-04-30 06:02:00,214.0,81230.0,2.082020e+14,3.365069e+10,3.548310e+13,D20434A,...,12784914.0,12784914.0,12784914.0,12784914.0,81230.0,42376.0,41915.0,57628851.0,631804110.0,"(72290.0, 86399.0]"
149997,7.349880e+18,2019-04-29 07:28:00,483.0,2019-04-30 10:41:00,187.0,97970.0,2.082020e+14,3.366345e+10,3.566050e+13,D10223C,...,4415361.0,4415361.0,4415361.0,4415361.0,97970.0,17264.0,16759.0,39135081.0,705783925.0,"(86400.0, 101468.0]"
149998,1.311450e+19,2019-04-29 07:28:00,283.0,2019-04-30 10:46:00,810.0,98249.0,2.082020e+14,3.362189e+10,3.572120e+13,T51102A,...,6585469.0,6585469.0,6585469.0,6585469.0,98249.0,25003.0,28640.0,34912224.0,627698629.0,"(86400.0, 101468.0]"


In [4]:
# create a df cleaner instance
cleaner = dataCleaner(df)
visualizer = dataVisualizer()

Data cleaner in action.
Data visualizer in action.


# User engagement analysis

* Tracking the user engagement based on session frequency, duration of session, and total traffic (UL+DL) on the network

In [6]:
# Aggregating the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric
user_engagement_df = df[['MSISDN/Number', 'Bearer Id', 'Dur. (ms)', 'total_data']]

user_engagement_df = user_engagement_df.groupby(
    'MSISDN/Number').agg({'Bearer Id': 'count', 'Dur. (ms)': 'sum', 'total_data': 'sum'})
user_engagement_df = user_engagement_df.rename(
    columns={'Bearer Id': 'XDR Sessions'})
user_engagement_df

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.360100e+10,1,116720.0,8.786906e+08
3.360100e+10,1,181230.0,1.568596e+08
3.360100e+10,1,134969.0,5.959665e+08
3.360101e+10,1,49878.0,4.223207e+08
3.360101e+10,2,37104.0,1.457411e+09
...,...,...,...
3.379000e+10,1,8810.0,7.146416e+08
3.379000e+10,1,140988.0,4.803073e+08
3.197020e+12,1,86399.0,2.321240e+08


In [9]:
user_engagement_df['XDR Sessions'].value_counts()

1       77730
2       21452
3        4422
4        1950
5         938
6         208
7          75
8          34
9          17
10         12
12          5
11          5
17          2
15          2
18          1
16          1
1066        1
13          1
Name: XDR Sessions, dtype: int64

In [10]:
#  top 10 customers per engagement metric - XDR sessions
user_engagement_df.nlargest(10, 'XDR Sessions')

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.366371e+10,1066,63940818.0,5.302862e+11
3.362632e+10,18,1785419.0,7.951348e+09
3.361489e+10,17,1678397.0,8.826326e+09
3.362578e+10,17,1599052.0,8.474657e+09
3.365973e+10,16,1905478.0,7.680848e+09
3.367588e+10,15,1745131.0,7.891111e+09
3.376054e+10,15,1651084.0,8.495067e+09
3.366716e+10,13,1209703.0,5.618394e+09
3.360313e+10,12,1109736.0,4.976195e+09


* These are the top 10 users per the XDR session engagement metric

In [13]:
#  top 10 customers per engagement metric - Durations
user_engagement_df.nlargest(10, 'Dur. (ms)')

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.366371e+10,1066,63940818.0,5.302862e+11
3.365973e+10,16,1905478.0,7.680848e+09
3.362632e+10,18,1785419.0,7.951348e+09
3.365936e+10,11,1747994.0,4.658437e+09
3.367588e+10,15,1745131.0,7.891111e+09
3.361489e+10,17,1678397.0,8.826326e+09
3.376054e+10,15,1651084.0,8.495067e+09
3.362578e+10,17,1599052.0,8.474657e+09
3.378632e+10,12,1427807.0,5.622232e+09


* These are the top 10 users per the duration of session engagement metric

In [12]:
#  top 10 customers per engagement metric - total data volume
user_engagement_df.nlargest(10, 'total_data')

,XDR Sessions,Dur. (ms),total_data
MSISDN/Number,,,
3.366371e+10,1066,63940818.0,5.302862e+11
3.361489e+10,17,1678397.0,8.826326e+09
3.376054e+10,15,1651084.0,8.495067e+09
3.362578e+10,17,1599052.0,8.474657e+09
3.362632e+10,18,1785419.0,7.951348e+09
3.367588e+10,15,1745131.0,7.891111e+09
3.365973e+10,16,1905478.0,7.680848e+09
3.366646e+10,11,1168703.0,7.308501e+09
3.376041e+10,12,1248477.0,7.112835e+09


* These are the top 10 users per the total data (UL + DL) engagement metric